### Q3

In [2]:
import kagglehub
import os

path = kagglehub.dataset_download("gopalbhattrai/pascal-voc-2012-dataset")

os.listdir(path)

100%|██████████| 3.52G/3.52G [17:22<00:00, 3.63MB/s] 

Extracting files...


['VOC2012_test', 'VOC2012_train_val']

In [36]:
# os.listdir(f'{path}/VOC2012_train_val/VOC2012_train_val')
annotations = f'{path}/VOC2012_train_val/VOC2012_train_val/Annotations'
images = f'{path}/VOC2012_train_val/VOC2012_train_val/JPEGImages'

In [ ]:
os.listdir(annotations)[:5], os.listdir(images)[:5]

(['2007_000027.xml',
  '2007_000032.xml',
  '2007_000033.xml',
  '2007_000039.xml',
  '2007_000042.xml'],
 ['2007_000027.jpg',
  '2007_000032.jpg',
  '2007_000033.jpg',
  '2007_000039.jpg',
  '2007_000042.jpg'])

In [91]:
def xml_to_label(xml):
    import xml.etree.ElementTree as ET

    tree = ET.parse(xml)
    root = tree.getroot()

    bbox = {}
    for child in root:
        if child.tag != 'object': continue
        for child2 in child:
            if child2.tag != 'bndbox': continue
            for child3 in child2:
                bbox[child3.tag] = int(child3.text)
    return bbox['xmin'], bbox['xmax'], bbox['ymin'], bbox['ymax']

In [75]:
import tensorflow as tf
import keras


X = keras.preprocessing.image_dataset_from_directory(
    images,
    seed=42,
    label_mode=None,
    batch_size=32
)

y = tf.data.Dataset.list_files(f"{annotations}/*.xml", seed=42)

Found 17125 files.


In [76]:
dataset = tf.data.Dataset.zip(X, y)

In [77]:
for x, y in dataset:
    print(bytes.decode(y.numpy()))
    break

D:\ML Env Paths\.cache\kagglehub\datasets\gopalbhattrai\pascal-voc-2012-dataset\versions\1\VOC2012_train_val\VOC2012_train_val\Annotations\2010_000661.xml


In [78]:
def preprocess(image):
    image = tf.image.resize(image, size=(300,300))
    image = tf.image.per_image_standardization(image)
    return image

In [79]:
# train on subset
dataset = dataset.take(20)

In [85]:
def tf_xml_to_label(y):
    return tf.py_function(lambda x: xml_to_label(bytes.decode(x.numpy())), inp=[y], Tout=[tf.int64, tf.int64, tf.int64, tf.int64])

In [92]:
processed_dataset = dataset.map(lambda x, y: (preprocess(x), tf_xml_to_label(y)))

In [93]:
for x, y in processed_dataset:
    print(y)
    break

tf.Tensor([ 74 239 104 375], shape=(4,), dtype=int64)


In [96]:
processed_dataset = processed_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [106]:
import tensorflow_hub as hub


pretrained = hub.KerasLayer("https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1", input_shape=(300,300,3), trainable=False)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [109]:
model = keras.Sequential([
    keras.layers.Lambda(pretrained),
    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dense(4, activation='relu'),
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-2),
    loss=keras.losses.MeanSquaredError(),
    metrics=[keras.metrics.MeanSquaredError()]
)

In [105]:
len(processed_dataset)

20

In [110]:
test_dataset = processed_dataset.take(5)
train_dataset = processed_dataset.skip(5)

In [111]:
model.fit(train_dataset, validation_data=test_dataset)

ValueError: Exception encountered when calling layer 'keras_layer_2' (type KerasLayer).

KerasLayer output does not contain the output key default (available: dict_keys(['detection_scores', 'detection_class_names', 'detection_class_entities', 'detection_boxes', 'detection_class_labels'])).

Call arguments received by layer 'keras_layer_2' (type KerasLayer):
  • inputs=tf.Tensor(shape=(None, 300, 300, 3), dtype=float32)
  • training=None